In [2]:
import tensorflow as tf
import cv2
import numpy as np
import data
import time
import matplotlib.pyplot as plt
early_stopping = None

In [3]:
image_size=128
train_path='/Users/abhiyendragahlot/Downloads/Convolutional_Neural_Networks/dataset/training_set'
classes=['dogs','cats']
data=data.read_train_sets(train_path=train_path,image_size=image_size,validation_size=0.16,classes=classes)

Reading training images
Loading dogs files (Index: 0)
Loading cats files (Index: 1)


In [117]:
def weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.05))

In [118]:
def biases(num):
    return tf.Variable(tf.constant(0.05,shape=[num]))

In [119]:
def conv(inpu,numinput,numoutfil):
    shape=[3,3,numinput,numoutfil]
    w=weights(shape)
    b=biases(numoutfil)
    laya=tf.nn.conv2d(input=inpu,filter=w,strides=[1,1,1,1],padding='SAME')
    laya=laya+b
    laya=tf.nn.max_pool(value=laya,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    laya=tf.nn.relu(laya)
    return laya

In [120]:
def flatten(inpu):
    shape=inpu.get_shape()
    a = int(shape[1]*shape[2]*shape[3])
    
    # Reshpe the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # the total size of the tensor is unchanged from the reshaping.
    laya = tf.reshape(inpu, [-1, a])

    return laya,a

In [121]:
def fclayer(inpu,no_inputs,no_neurons,relu=True):
    w=weights([no_inputs,no_neurons])
    b=biases(no_neurons)
    ans=(tf.matmul(inpu,w)+b)
    if relu:
        ans=tf.nn.relu(ans)
    return ans

In [122]:
x=tf.placeholder(tf.float32,shape=[None,128*128*3])
x_image = tf.reshape(x, [-1, 128, 128, 3])

In [123]:
ytrue=tf.placeholder(tf.float32,shape=[None,2])

In [124]:
y_true_cls=tf.arg_max(ytrue,dimension=1)

In [125]:
layer=conv(inpu=x_image,numinput=3,numoutfil=32)
layer=conv(inpu=layer,numinput=32,numoutfil=32)
layer=conv(inpu=layer,numinput=32,numoutfil=64)

In [126]:
layer,numfeatures=flatten(layer)

In [127]:
layer=fclayer(inpu=layer,no_inputs=numfeatures,no_neurons=128,relu=True)

In [128]:
layer=fclayer(inpu=layer,no_inputs=128,no_neurons=2,relu=False)


In [129]:
ypred=tf.nn.softmax(layer)

In [130]:
y_pred_cls = tf.argmax(ypred, dimension=1)




cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer,
                                                        labels=ytrue)





cost = tf.reduce_mean(cross_entropy)





optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)





correct_prediction = tf.equal(y_pred_cls, y_true_cls)





accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





session=tf.Session()





session.run(tf.global_variables_initializer())





train_batch_size = 32



In [131]:
def print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    # Calculate the accuracy on the training-set.
    
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
   # msg = "Epoch {0} ---, Validation Accuracy: {2:>6.1%}, Validation Loss: {3:.3f}"
    print(epoch + 1,val_acc, val_loss)


In [132]:
total_iterations = 0

def optimize(num_iterations):
    # Ensure we update the global variable rather than a local copy.
    global total_iterations

    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    best_val_loss = float("inf")
    patience = 0

    for i in range(total_iterations,
                   total_iterations + num_iterations):

        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)

        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, flattened image shape]

        #x_batch = x_batch.reshape(train_batch_size, img_size_flat)
        #x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)
        x_batch = x_batch.reshape(train_batch_size, 128*128*3)
        x_valid_batch = x_valid_batch.reshape(train_batch_size, 128*128*3)

        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        feed_dict_train = {x: x_batch,
                           ytrue: y_true_batch}
        
        feed_dict_validate = {x: x_valid_batch,
                              ytrue: y_valid_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)
        

        # Print status epoch (defined as full pass through training dataset).
        if i % int(data.train.num_examples/32) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_validate)
            epoch = int(i / int(data.train.num_examples/32))
            
            print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss)
            
            if early_stopping:    
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience = 0
                else:
                    patience += 1

                if patience == early_stopping:
                    break

    # Update the total number of iterations performed.
    total_iterations += num_iterations

    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print th time-usage.
    print("Time elapsed: " + str(timedelta(seconds=int(round(time_dif)))))



In [ ]:
optimize(9000)